In [1]:
import sys, time, random, math, cv2, os
import numpy as np

sys.path.insert(0, '../src')
from robot import Robot

In [2]:
N_STATE_GTG = 120
actions_gtg = np.array([(3,3), (1,-1), (-1,1)])

dimensions = (N_STATE_GTG, len(actions_gtg))

path = "Q_matrix_gtg.txt"
on_disk = path in os.listdir()

if on_disk:
    f = open(path, "r")
    
    t = f.read().split()
    iterations = int(t[0])
    random_rate = float(t[1])
    learning_rate = float(t[2])
    
    Q = np.reshape(np.array([float(i) for i in t[3:]]), dimensions)

    f.close()
else:
    iterations = 0
    random_rate = 0.9
    learning_rate = 0.6
    Q = np.zeros(dimensions)

In [3]:
def cicle(ang):
    return (ang + 2 * math.pi) % (2*math.pi)

def get_distance_from_goal(a,b):
    return np.linalg.norm(a-b)

def get_angular_distance(v1,v2):

    v1_aux = v1 / np.linalg.norm(v1)
    v2_aux = v2 / np.linalg.norm(v2)

    theta = np.arccos(np.clip(np.dot(v1_aux, v2_aux), -1.0, 1.0))
    rot_theta = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    theta = theta*180.0/math.pi
    theta = (360.0+theta)%360.0

    if abs(np.dot(np.dot(rot_theta,v2_aux),v1_aux)-1.0) > 1e-2:
        theta = 360.0 - theta

    return theta

def get_state_go_to_goal(current_position, current_angle, goal):
    angle_distance = get_angular_distance(goal - current_position, np.array([math.cos(current_angle),math.sin(current_angle)]))
    if angle_distance <= (180 / N_STATE_GTG) or angle_distance >= (360 - (180 / N_STATE_GTG)):
        r = 0
    else:
        r = (angle_distance - (180 / N_STATE_GTG)) / (360 / N_STATE_GTG) + 1
    return int(r)

def get_action_go_to_goal(Q, state, random_rate):
    a = random.random()
    if a < random_rate:
        i = random.randint(0, len(actions_gtg) - 1)
    else:
        i = 0
        for j in range(len(actions_gtg)):
            if Q[state][j] > Q[state][i]:
                i = j
    return i
    
def get_reward(current_position, current_angle, new_position, new_angle, goal):
    angle_distance = get_angular_distance(goal - current_position, np.array([math.cos(current_angle),math.sin(current_angle)]))
    new_angle_distance = get_angular_distance(goal - new_position, np.array([math.cos(new_angle),math.sin(new_angle)]))
    if angle_distance > 180: angle_distance = 360 - angle_distance
    if new_angle_distance > 180: new_angle_distance = 360 - new_angle_distance
    print("nad: " + str(new_angle_distance))
    print("oad: " + str(angle_distance))
    delta_angle = angle_distance - new_angle_distance

    distance = get_distance_from_goal(current_position, goal)
    new_distance = get_distance_from_goal(new_position, goal)
    print("nd: " + str(new_distance))
    print("od: " + str(distance))
    delta_distance = distance - new_distance

    return delta_angle + 50 * delta_distance
    

In [ ]:
robot = Robot("#0")
goal = np.array([13,0])

max_iterations = 1000

goal = [(13,0), (-13,0)]
i = 0

while robot.get_connection_status() != -1 and iterations < max_iterations:
    current_position = np.array(robot.get_current_position()[:-1])
    current_angle = robot.get_current_orientation()[2]
    
    state = get_state_go_to_goal(current_position, current_angle, goal[i])
    
    ac = get_action_go_to_goal(Q, state, random_rate)
    
    robot.set_left_velocity(actions_gtg[ac][0])
    robot.set_right_velocity(actions_gtg[ac][1])
    time.sleep(3)
    robot.set_left_velocity(0)
    robot.set_right_velocity(0)

    new_position = np.array(robot.get_current_position()[:-1])
    new_angle = robot.get_current_orientation()[2]
    
    reward = get_reward(current_position, current_angle, new_position, new_angle, goal[i])
    
    print("state: " + str(state) + " action: " + str(ac) + " reward: " + str(reward))
    
    best_q = np.amax(Q[state])

    Q[state][ac] = learning_rate * (reward + 0.99 * best_q - Q[state][ac])
    
    if get_distance_from_goal(new_position, goal[i]):
        i = (i + 1) % 2
    
    random_rate = max(0.1, random_rate * 0.9995)
    learning_rate = max(0.1, learning_rate * 0.9999)

    f = open(path, "w")
    f.write(str(iterations) + "\n")
    f.write(str(random_rate) + "\n")
    f.write(str(learning_rate) + "\n")

    for q in Q:
        for j in q:
            f.write(str(j) + " ")
        f.write("\n")
    f.close()

    iterations += 1


Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1#0 connected.
 Pioneer_p3dx_ultrasonicSensor2#0 connected.
 Pioneer_p3dx_ultrasonicSensor3#0 connected.
 Pioneer_p3dx_ultrasonicSensor4#0 connected.
 Pioneer_p3dx_ultrasonicSensor5#0 connected.
 Pioneer_p3dx_ultrasonicSensor6#0 connected.
 Pioneer_p3dx_ultrasonicSensor7#0 connected.
 Pioneer_p3dx_ultrasonicSensor8#0 connected.
 Pioneer_p3dx_ultrasonicSensor9#0 connected.
 Pioneer_p3dx_ultrasonicSensor10#0 connected.
 Pioneer_p3dx_ultrasonicSensor11#0 connected.
 Pioneer_p3dx_ultrasonicSensor12#0 connected.
 Pioneer_p3dx_ultrasonicSensor13#0 connected.
 Pioneer_p3dx_ultrasonicSensor14#0 connected.
 Pioneer_p3dx_ultrasonicSensor15#0 connected.
 Pioneer_p3dx_ultrasonicSensor16#0 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.
nad: 48.93030130876002
oad: 0.17153151760362562
nd: 25.01370740669083
od: 25.025442012600774
state: 0 action: 2 reward: -48.172039495

nad: 57.50062100193003
oad: 11.141846851306752
nd: 21.196210845333255
od: 21.27010170336723
state: 4 action: 1 reward: -42.66423124892444
nad: 89.82128775691058
oad: 146.92346620158992
nd: 5.218872191009742
od: 5.178914892828138
state: 71 action: 1 reward: 55.10431353559913
nad: 171.77846690954652
oad: 114.42535493393086
nd: 21.12613816628889
od: 21.1530206001542
state: 38 action: 1 reward: -56.00899028235014
nad: 24.79524508329291
oad: 32.786434009465154
nd: 4.824763059709918
od: 5.257692857371589
state: 109 action: 0 reward: 29.637678809255817
nad: 131.2350191037653
oad: 177.97250960444137
nd: 21.6699860286753
od: 21.603039989358077
state: 59 action: 1 reward: 43.390188534814925
nad: 24.549457685252946
oad: 21.152121193126106
nd: 4.765075525306254
od: 4.779883841594384
state: 7 action: 2 reward: -2.6569206777203256
nad: 134.0163806377662
oad: 177.47168253009204
nd: 21.67733153833028
od: 21.666591728793982
state: 59 action: 1 reward: 42.91831141551101
nad: 75.10813898581091
oad: 18.41

nad: 38.24498800268361
oad: 21.604448405350183
nd: 20.44857410709194
od: 20.46634103013111
state: 7 action: 2 reward: -15.752193445374822
nad: 97.35473220823684
oad: 104.56160701389211
nd: 6.637633019273128
od: 6.5841954759281665
state: 35 action: 0 reward: 4.534997638407193
nad: 3.372774921151745
oad: 50.421400448945235
nd: 20.120659620643654
od: 20.14272020725027
state: 103 action: 1 reward: 48.15165485812435
nad: 155.73574880576882
oad: 145.28322544046813
nd: 6.606433622781636
od: 6.616119712574976
state: 48 action: 1 reward: -9.968218875633683
nad: 113.45608981557126
oad: 55.10025317427238
nd: 20.064082675005896
od: 20.105996656078535
state: 18 action: 1 reward: -56.26013758766692
nad: 38.67873654803947
oad: 96.9989760852394
nd: 6.676676226787785
od: 6.63279282148833
state: 88 action: 1 reward: 56.12606927222721
nad: 127.74652710208983
oad: 171.88823106205382
nd: 20.04797614124139
od: 20.028486912385794
state: 57 action: 2 reward: 43.167242517184285
nad: 96.71948547128716
oad: 82.6